In [49]:
import json

In [5]:
import urllib.request as r
from bs4 import BeautifulSoup as bs
import json

url = r.urlopen("https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json")
content = url.read()
soup = bs(content)

In [1]:
!wget https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json

--2023-12-01 09:35:09--  https://raw.githubusercontent.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM/main/data/alpaca_gpt4_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43379276 (41M) [text/plain]
Saving to: ‘alpaca_gpt4_data.json.1’

alpaca_gpt4_data.js 100%[===================>]  41.37M  --.-KB/s    in 0.1s    

2023-12-01 09:35:11 (340 MB/s) - ‘alpaca_gpt4_data.json.1’ saved [43379276/43379276]



In [50]:
import json

dataset_file = "alpaca_gpt4_data.json"

with open(dataset_file, "r") as f:
    alpaca = json.load(f)

In [51]:
type(alpaca), alpaca[0:3], len(alpaca)

(list,
 [{'instruction': 'Give three tips for staying healthy.',
   'input': '',
   'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'},
  {'instruction': 'What are the three primary colors?',
   'input': '',
   'output': 'The three primary colors are red, blue, and yellow. These

In [52]:
import random

seed = 42

random.seed(seed)
random.shuffle(alpaca) 

In [53]:
train_dataset = alpaca[:-1000]
eval_dataset = alpaca[-1000:]

In [54]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n").format_map(row)

def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

def create_alpaca_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

In [55]:
train_prompts = [create_alpaca_prompt(row) for row in train_dataset]
eval_prompts = [create_alpaca_prompt(row) for row in eval_dataset]

In [56]:
def pad_eos(ds):
    EOS_TOKEN = "</s>"
    return [f"{row['output']}{EOS_TOKEN}" for row in ds]

# add <s> token to prompts
# train_prompts = [f"<s>{prompt}" for prompt in train_prompts]
# eval_prompts = [f"<s>{prompt}" for prompt in eval_prompts]

In [57]:
train_outputs = pad_eos(train_dataset)
eval_outputs = pad_eos(eval_dataset)

In [58]:
train_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(train_prompts, train_outputs)]
eval_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(eval_prompts, eval_outputs)]

In [59]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [60]:
model_id = "TheBloke/Llama-2-7B-GPTQ"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
tokenizer_gpt = AutoTokenizer.from_pretrained("gpt2")
tokenizer_llama.pad_token = tokenizer_llama.eos_token
tokenizer_gpt.pad_token = tokenizer_gpt.eos_token

tokenizer_config.json: 100%|██████████| 727/727 [00:00<00:00, 4.26MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 15.7MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 40.7MB/s]
special_tokens_map.json: 100%|██████████| 411/411 [00:00<00:00, 3.16MB/s]


In [64]:
print(tokenizer_llama.decode(tokenizer_llama.encode(f"My experiments are going strong."), skip_special_tokens=False))
print(tokenizer_gpt.encode(f"{tokenizer_gpt.bos_token}My experiments are going strong.{tokenizer_gpt.bos_token}"))

<s> My experiments are going strong.
[50256, 3666, 10256, 389, 1016, 1913, 13, 50256]


In [48]:
tokenizer_gpt.decode([50256])

'<|endoftext|>'

In [12]:
print(train_dataset[0]["example"])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Describe an example of a time you used influence in a positive way

### Response:
As an AI assistant, I do not have personal experiences, but I can provide an example. One instance where I used my influence in a positive way was when a user was feeling highly stressed and overwhelmed about an upcoming job interview. They confided in me, expressing their self-doubts and fears of failure. Recognizing the power of positive reinforcement and encouragement, I drew upon my resources to provide the user with uplifting and motivational messages. I reminded them of their strengths and past accomplishments, and suggested coping strategies such as visualization and practicing positive self-talk. Furthermore, I helped them prepare for the interview by offering practice questions, tips on body language, and advice on how to effectively communicate their skills and experiences.

In [14]:
tokenizer.encode('</s>')

[1, 2]

In [65]:
max_sequence_len = 1024

def pack(dataset, max_seq_len=max_sequence_len):
    tkds_ids = tokenizer([s["example"] for s in dataset])["input_ids"]
    
    all_token_ids = []
    for tokenized_input in tkds_ids:
        all_token_ids.extend(tokenized_input + [tokenizer.eos_token_id])
    
    print(f"Total number of tokens: {len(all_token_ids)}")
    packed_ds = []
    for i in range(0, len(all_token_ids), max_seq_len+1):
        input_ids = all_token_ids[i : i + max_seq_len+1]
        if len(input_ids) == (max_seq_len+1):
            packed_ds.append({"input_ids": input_ids[:-1], "labels": input_ids[1:]})  # this shift is not needed if using the model.loss
    return packed_ds


In [66]:
train_ds_packed = pack(train_dataset)
eval_ds_packed = pack(eval_dataset)
len(train_ds_packed)

Total number of tokens: 11543478
Total number of tokens: 224900


11261

In [67]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

torch.manual_seed(seed)
batch_size = 1  # I have an A100 GPU with 40GB of RAM 😎

train_dataloader = DataLoader(
    train_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator, # we don't need any special collator 😎
)

eval_dataloader = DataLoader(
    eval_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    shuffle=False,
)

In [68]:
b = next(iter(train_dataloader))
b["input_ids"].shape, b["labels"].shape

(torch.Size([1, 1024]), torch.Size([1, 1024]))

In [26]:
b["input_ids"][0]

tensor([    1, 13866,   338,  ...,  3661,  2158, 29889])

In [69]:
tokenizer.decode(b["input_ids"][0])[:2000]

'<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe an example of a time you used influence in a positive way\n\n### Response:\nAs an AI assistant, I do not have personal experiences, but I can provide an example. One instance where I used my influence in a positive way was when a user was feeling highly stressed and overwhelmed about an upcoming job interview. They confided in me, expressing their self-doubts and fears of failure. Recognizing the power of positive reinforcement and encouragement, I drew upon my resources to provide the user with uplifting and motivational messages. I reminded them of their strengths and past accomplishments, and suggested coping strategies such as visualization and practicing positive self-talk. Furthermore, I helped them prepare for the interview by offering practice questions, tips on body language, and advice on how to effectively communicate their skills and e

In [20]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>'}

In [18]:
tokenizer.decode(b["labels"][0])[:250]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe an example of a time you used influence in a positive way\n\n### Response:\nAs an AI assistant, I do not have personal e'

In [70]:
from types import SimpleNamespace

gradient_accumulation_steps = 2

config = SimpleNamespace(
    model_id="TheBloke/Llama-2-7B-GPTQ",
    dataset_name="alpaca-gpt4",
    precision="bf16",  # faster and better than fp16, requires new GPUs
    n_freeze=24,  # How many layers we don't train, LLama 7B has 32.
    lr=1e-4,
    n_eval_samples=10, # How many samples to generate on validation
    max_seq_len=max_sequence_len, # Lenght of the sequences to pack
    epochs=3,  # we do 3 pasess over the dataset.
    gradient_accumulation_steps=gradient_accumulation_steps,  # evey how many iterations we update the gradients, simulates larger batch sizes
    batch_size=batch_size,  # what my GPU can handle, depends on how many layers are we training  
    log_model=False,  # upload the model to W&B?
    gradient_checkpointing = True,  # saves even more memory
    freeze_embed = True,  # why train this? let's keep them frozen ❄️
    seed=seed,
)

config.total_train_steps = config.epochs * len(train_dataloader) // config.gradient_accumulation_steps

In [71]:
print(f"We will train for {config.total_train_steps} steps and evaluate every epoch")

We will train for 16891 steps and evaluate every epoch


In [74]:
model = AutoModelForCausalLM.from_pretrained(
    config.model_id,
    device_map=0,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)

ImportError: Loading a GPTQ quantized model requires optimum (`pip install optimum`) and auto-gptq library (`pip install auto-gptq`)

In [25]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

params, trainable_params = param_count(model)

Total params: 774.03M, Trainable: 774.03M


In [26]:
from transformers import get_cosine_schedule_with_warmup

optim = torch.optim.Adam(model.parameters(), lr=config.lr, betas=(0.9,0.99), eps=1e-5)
scheduler = get_cosine_schedule_with_warmup(
    optim,
    num_training_steps=config.total_train_steps,
    num_warmup_steps=config.total_train_steps // 10,
)

In [27]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

In [28]:
from types import SimpleNamespace
from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained(config.model_id)
test_config = SimpleNamespace(
    max_new_tokens=256,
    gen_config=gen_config)

In [29]:
def generate(prompt, max_new_tokens=test_config.max_new_tokens, gen_config=gen_config):
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
    with torch.inference_mode():
        output = model.generate(tokenized_prompt, 
                            max_new_tokens=max_new_tokens, 
                            generation_config=gen_config,
                            temperature=0.7,
                            num_beams=5,
                            no_repeat_ngram_size=2,
                            early_stopping=True,
                            do_sample=True,
                            pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

In [30]:
prompt = eval_dataset[14]["prompt"]
print(prompt + generate(prompt, 128))

<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Add an adverb to the sentence to make it sound more interesting

### Input:
She cooked the chicken.

### Response:
... The chicken was cooked in the oven.


This is a very simple example, but it illustrates how you can use adverbs to modify the meaning of a sentence. For more complex sentences, you'll need to think about the context of the sentences you're writing, and how they relate to each other.


In [31]:
def to_gpu(tensor_dict):
    return {k: v.to('cuda') for k, v in tensor_dict.items()}

class Accuracy:
    "A simple Accuracy function compatible with HF models"
    def __init__(self):
        self.count = 0
        self.tp = 0.
    def update(self, logits, labels):
        logits, labels = logits.argmax(dim=-1).view(-1).cpu(), labels.view(-1).cpu()
        tp = (logits == labels).sum()
        self.count += len(logits)
        self.tp += tp
        return tp / len(logits)
    def compute(self):
        return self.tp / self.count

In [32]:
from tqdm.auto import tqdm

In [33]:
@torch.no_grad()
def validate():
    model.eval();
    eval_acc = Accuracy()
    loss, total_steps = 0., 0
    for step, batch in enumerate(pbar:=tqdm(eval_dataloader, leave=False)):
        pbar.set_description(f"doing validation")
        batch = to_gpu(batch)
        total_steps += 1
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss += loss_fn(out.logits, batch["labels"])  # you could use out.loss and not shift the dataset
        eval_acc.update(out.logits, batch["labels"])
    # we log results at the end
    model.train();

In [34]:
# Training
acc = Accuracy()
model.train()
train_step = 0
pbar = tqdm(total=config.total_train_steps)
for epoch in range(config.epochs):
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = to_gpu(batch)
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss = loss_fn(out.logits, batch["labels"]) / config.gradient_accumulation_steps  # you could use out.loss and not shift the dataset  
            loss.backward()
        if step%config.gradient_accumulation_steps == 0:
            optim.step()
            scheduler.step()
            optim.zero_grad(set_to_none=True)
            train_step += 1
            pbar.update(1)
            pbar.set_description(f"loss: {loss.item():.4f}")
    validate()    

 12%|█▏        | 1199/10157 [06:49<50:57,  2.93it/s]:47,  1.46it/s]


KeyboardInterrupt: 

In [35]:
prompt = eval_dataset[14]["prompt"]
print(prompt + generate(prompt, 128))

<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Add an adverb to the sentence to make it sound more interesting

### Input:
She cooked the chicken.

### Response:
The chicken was cooked in a delicious way. She cooked it in her own kitchen, and it was truly delicious.</s></li><input type="hidden" name="comment" value="###" /></input></div>


<div class="col-xs-12 col-md-6"> <h1>Welcome to Memrise!</h2> <p>Here you can learn about topics related to computer science, from beginner to advanced. Use the menus at the top of the page to choose a topic to start with, or browse our vast library of courses to find something that interests you.</


In [6]:
prompts = [create_prompt(row) for row in alpaca]

In [7]:
dataset = [{"prompt":s, "output":t, "example": s+t} for s, t in zip(prompts, outputs)]

In [8]:
dataset[0]

{'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.</s>',
 'example': 'Below is an i

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/rkd43/Teaching/large-language-models/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model_id = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [11]:
import random
random.shuffle(dataset)

train_dataset = dataset[:-1000]
eval_dataset = dataset[-1000:]

In [12]:
train

NameError: name 'train' is not defined

In [13]:
max_seq_len = 1024


def pack(dataset, max_seq_len=1024):
    tkds_ids = tokenizer([s["example"] for s in dataset])["input_ids"]
    
    all_token_ids = []
    for tokenized_input in tkds_ids:
        all_token_ids.extend(tokenized_input + [tokenizer.eos_token_id])
    
    packed_ds = []
    for i in range(0, len(all_token_ids), max_seq_len+1):
        input_ids = all_token_ids[i : i + max_seq_len+1]
        if len(input_ids) == (max_seq_len+1):
            packed_ds.append({"input_ids": input_ids[:-1], "labels": input_ids[1:]})  # < --- ‼️ ⛔️
	    # if you use the model.output.loss you don't need to shift, it is done for you!
    return packed_ds


train_ds_packed = pack(train_dataset)
eval_ds_packed = pack(eval_dataset)

Token indices sequence length is longer than the specified maximum sequence length for this model (1149 > 1024). Running this sequence through the model will result in indexing errors


In [14]:
# save the packed datasets as json files
with open("train_ds_packed.json", "w") as f:
    json.dump(train_ds_packed, f)

with open("eval_ds_packed.json", "w") as f:
    json.dump(eval_ds_packed, f)
    

In [15]:
from datasets import load_from_disk

In [16]:
from torch.utils.data import DataLoader
from transformers import default_data_collator


batch_size = 4  # I have an A100 GPU with 40GB of RAM 😎


train_dataloader = DataLoader(
    train_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator, # we don't need any special collator 😎
)


eval_dataloader = DataLoader(
    eval_ds_packed,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    shuffle=False,
)

In [17]:
batch = next(iter(train_dataloader))

input_text = tokenizer.decode(batch["input_ids"][0])
target_text = tokenizer.decode(batch["labels"][0])

print("Input text:\n", input_text)

Input text:
 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Design a logo for a bakery based on provided details.

### Input:
The bakery’s name is “Noxious Treats” and they specialize in desserts.

### Response:
1. Start by making a design for the table including the dimensions and the shape of the tabletop, legs, and any additional features. 

2. Once you have the design, gather all the materials you will need: Wood, paint, glue, nails, and screws.

3. Cut the wood according to the dimensions in your design, and sand the edges to make them smooth.

4. Assemble the tabletop by gluing together the pieces of wood and securing them with nails or screws. Make sure the tabletop is level and let the glue dry completely.

5. Attach the legs to the tabletop using glue and screws or nails. You may need to use extra pieces of wood for support to ensure that the legs ar

In [18]:
import torch

model = AutoModelForCausalLM.from_pretrained(
    'gpt2',
    device_map=0,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)


In [19]:
from transformers import get_cosine_schedule_with_warmup

In [20]:
from types import SimpleNamespace

max_seq_len = 1024

gradient_accumulation_steps = 32 // batch_size

config = SimpleNamespace(
    model_id='gpt2',
    dataset_name="alpaca-gpt4",
    precision="bf16",  # faster and better than fp16, requires new GPUs
    n_freeze=24,  # How many layers we don't train, LLama 7B has 32.
    lr=2e-4,
    n_eval_samples=10, # How many samples to generate on validation
    max_seq_len=max_seq_len, # Length of the sequences to pack
    epochs=3,  # we do 3 pasess over the dataset.
    gradient_accumulation_steps=gradient_accumulation_steps,  # evey how many iterations we update the gradients, simulates larger batch sizes
    batch_size=batch_size,  # what my GPU can handle, depends on how many layers are we training  
    log_model=False,  # upload the model to W&B?
    mom=0.9, # optim param
    gradient_checkpointing = True,  # saves even more memory
    freeze_embed = True,  # why train this? let's keep them frozen ❄️
)


config.total_train_steps = config.epochs * len(train_dataloader) // config.gradient_accumulation_steps


In [20]:
# freeze layers (disable gradients)
for param in model.parameters(): param.requires_grad = False
for param in model.lm_head.parameters(): param.requires_grad = True
for param in model.model.layers[config.n_freeze:].parameters(): param.requires_grad = True

AttributeError: 'GPT2LMHeadModel' object has no attribute 'model'

In [39]:
if config.freeze_embed:
    model.model.embed_tokens.weight.requires_grad_(False)

In [21]:
# save more memory
if config.gradient_checkpointing:
    model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})

In [22]:
optim = torch.optim.Adam(model.parameters(), lr=config.lr, betas=(0.9,0.99), eps=1e-5)
scheduler = get_cosine_schedule_with_warmup(
    optim,
    num_training_steps=config.total_train_steps,
    num_warmup_steps=config.total_train_steps // 10,
)


def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

In [23]:
from transformers import GenerationConfig
gen_config = GenerationConfig.from_pretrained(config.model_id)


def generate(prompt, max_new_tokens=100, gen_config=gen_config):
    with torch.inference_mode():
        tokenized_prompt = tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
        output = model.generate(tokenized_prompt, 
                            max_new_tokens=max_new_tokens, 
                            generation_config=gen_config)
    return tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)


In [24]:
@torch.no_grad()
def validate():
    model.eval();
    eval_acc = Accuracy()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = to_gpu(batch)
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss = loss_fn(out.logits, batch["labels"])  # you could use out.loss and not shift the dataset
        eval_acc.update(out.logits, batch["labels"])
    # we log results at the end
    model.train()

In [25]:
class Accuracy:
    "A simple Accuracy function compatible with HF models"
    def __init__(self):
        self.count = 0
        self.tp = 0.
    def update(self, logits, labels):
        logits, labels = logits.argmax(dim=-1).view(-1).cpu(), labels.view(-1).cpu()
        tp = (logits == labels).sum()
        self.count += len(logits)
        self.tp += tp
        return tp / len(logits)
    def compute(self):
        return self.tp / self.count

def to_gpu(tensor_dict):
    return {k: v.to('cuda') for k, v in tensor_dict.items()}

In [26]:
from tqdm.auto import tqdm

In [27]:
acc = Accuracy()

In [28]:
model.train()
train_step = 0
pbar = tqdm(total=config.total_train_steps)
for epoch in range(config.epochs):
    for step, batch in enumerate(train_dataloader):
        batch = to_gpu(batch)
        with torch.amp.autocast("cuda", dtype=torch.bfloat16):
            out = model(**batch)
            loss = loss_fn(out.logits, batch["labels"]) / config.gradient_accumulation_steps  # you could use out.loss and not shift the dataset  
            loss.backward()
        if step%config.gradient_accumulation_steps == 0:
            optim.step()
            scheduler.step()
            optim.zero_grad(set_to_none=True)
            train_step += 1
            pbar.update(1)
            pbar.set_description(f"loss: {loss.item():.4f}")
    validate()
pbar.close()

loss: 0.3654:   2%|▏         | 10/641 [00:23<26:25,  2.51s/it]

KeyboardInterrupt: 

In [36]:
# get next batch
batch = next(iter(eval_dataloader))

In [38]:
input_text = tokenizer.decode(batch["input_ids"][0])
target_text = tokenizer.decode(batch["labels"][0])

print("Input text:\n", input_text)

Input text:
 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Determine the name of the compound given the following chemical formula

### Input:
C6H12O6

### Response:
The chemical formula C6H12O6 represents the compound glucose.</s><|endoftext|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe a scene from a movie.

### Input:
The Shawshank Redemption

### Response:
The scene opens with a series of several establishing shots of the exterior of Shawshank Prison in the year of 1947. It's a drab and imposing fortress-like structure, surrounded by layers of fences topped with barbed wire. The camera then cuts to a shot of a bus full of new prisoners arriving at the prison. Among them is the protagonist, Andy Dufresne, handcuffed a

In [41]:
prompt = "QUESTION: What did you talk to President Kim about?"

output = generate(prompt, gen_config=gen_config)

In [42]:
print(prompt + "\n\n" + output)

QUESTION: What did you talk to President Kim about?



PRESIDENT KIM: Well, I think it was about the President's remarks. I think he talked about the President Trump's comments on the South Korean issue.

I think it was about the President's comments on the South Korean issue. I think he talked about the President's comments on the South Korean issue. I think it was about the President's comments on the South Korean issue. I think it was about the President's comments on the President Trump's comments on the South
